In [ ]:
!pip install nibabel
!pip install seaborn
!pip install openpyxl

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Input
from keras.layers import Convolution3D,Conv3D,Add,concatenate, MaxPool3D,add, Convolution2D,BatchNormalization, GlobalAveragePooling3D, ZeroPadding3D
from keras import backend as K
import cv2
import operator
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
import nibabel as nib
import pandas as pd
from scipy.ndimage import rotate
import random
import nibabel as nib
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
import nibabel as nib
from matplotlib import animation, rc
from scipy.ndimage import rotate
import glob
import os
from sklearn.model_selection import KFold, StratifiedKFold,train_test_split
from scipy.ndimage import zoom
from matplotlib.patches import PathPatch, Rectangle
from IPython.display import HTML

In [ ]:
# Enable TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Device:", tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print("Number of accelerators: ", strategy.num_replicas_in_sync)

# Model



In [ ]:
def define_model():
    model = Sequential()
    model.add(Conv3D(32,(3,3,3), activation='relu', kernel_initializer='he_uniform', padding='same',input_shape=(32,32,32,1)))
    model.add(BatchNormalization())
    model.add(Conv3D(32, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))

    
    model.add(Conv3D(64, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(64, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))

    model.add(Conv3D(256, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(256, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(8, activation='relu', kernel_initializer='he_uniform'))


    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_uniform'));
    return model

### Luna Load

In [ ]:
luna_df = pd.read_excel('/kaggle/input/spie-cropped-resampled-dataset/final_Luna16.xlsx')
column_luna = 'malignancy'
luna_series=luna_df['Series Uid'].tolist()

luna_folder = '/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled'  
luna = []
for i in luna_series:
    nii_file_path = os.path.join(luna_folder, f'{i}.nii')  # Assuming NII files have the extension '.nii'
    
    if os.path.exists(nii_file_path):
        luna.append(i)
luna=np.array(luna)

print(luna.shape)

### unlabeled load

In [ ]:
# Load the Excel file for unlabeled_df
unlabeled_df = pd.read_excel('/kaggle/input/semi-testtrainval/combined_correct_predictions.xlsx')
column_unlabeled = 'malignancy'
unlabeled_series = unlabeled_df['scan'].tolist()

unlabeled_folder = '/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled'
unlabeled = []
for i in unlabeled_series:
    nii_file_path = os.path.join(unlabeled_folder, f'{i}.nii')
    
    if os.path.exists(nii_file_path):
        unlabeled.append(i)
unlabeled = np.array(unlabeled)

print(unlabeled.shape)

In [ ]:
BS= 48 # batch size
E = 200 #epoch

In [ ]:
def aug_rotate(x):

    rotated_slices = []
    for slice_idx in range(x.shape[2]):
        slice_data = x[:, :, slice_idx]  # Extract a single slice
        rotated_slice = rotate(slice_data, angle=angles[j], reshape=False, mode='nearest')
        rotated_slices.append(rotated_slice)

    # Create a new NIfTI image from the rotated slices
    rotated_img_data = np.stack(rotated_slices, axis=-1)
    # Create a new NIfTI image using the header information from the original image
    rotated_nifti = nib.Nifti1Image(rotated_img_data, img.affine)
    
    return rotated_nifti

split

### Load Data

In [ ]:
# Specify the path to the Excel file
file_path = '/kaggle/input/semi-testtrainval/test_train_vai_set.xlsx'
# Load the Excel file
df = pd.read_excel(file_path)
# z_train

z_train = df.iloc[:, 2]    # Third column

#z_test
z_test_fixed = df.iloc[:, 0]  # First column
nan_count = z_test_fixed.isna().sum()
z_test = z_test_fixed.dropna()

#z_val
z_val_fixed = df.iloc[:, 1]  # Second column
nan_count_val = z_val_fixed.isna().sum()
z_val = z_val_fixed.dropna()


In [ ]:
datas_test = []
labels_test = []
scans_test = []

datas_train = []
labels_train = []
scans_train = []

datas_val = []
labels_val = []
scans_val = []
angles = [0, 45, 90, 135, 180, 225, 270, 315]


In [ ]:
print(z_train.shape)
print(z_train.head())

In [ ]:
for i in z_train:  # Assuming i is a value in the first column
    matching_rows_luna = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy']
    matching_rows_unlabeled = unlabeled_df.loc[unlabeled_df['scan'] == i, 'malignancy']
    if not matching_rows_luna.empty:
        lab = matching_rows_luna.values[0]
        img_files = glob.glob(os.path.join(
            "/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/",
            i + "*.nii"
        ))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            for j in range(len(angles)):
                rotated_nifti = aug_rotate(data)
                datas_train.append(rotated_nifti.get_fdata())
                labels_train.append(lab)

    elif not matching_rows_unlabeled.empty:
        lab = matching_rows_unlabeled.values[0]
        img_files = glob.glob(os.path.join(
            "/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/",
            i + "*.nii"
        ))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            for j in range(len(angles)):
                rotated_nifti = aug_rotate(data)
                datas_train.append(rotated_nifti.get_fdata())
                labels_train.append(lab)
    else:
        print(f"No matching Series Uid found for: {i}")
X_train = np.array(datas_train)
y_train = np.array(labels_train)


print(X_train.shape)

In [ ]:
for i in z_val:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_val.append(data)
        labels_val.append(lab)

for i in z_test:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_test.append(data)
        labels_test.append(lab)
            

X_test = np.array(datas_test)
y_test = np.array(labels_test)
X_val = np.array(datas_val)
y_val = np.array(labels_val)


X_train = np.array(datas_train)
y_train = np.array(labels_train)
X_test = np.array(datas_test)
y_test = np.array(labels_test)
X_val = np.array(datas_val)
y_val = np.array(labels_val)

print(y_train.shape, y_test.shape, y_val.shape)
print(X_train.shape, X_test.shape, X_val.shape)

model train

# HU

model

In [ ]:
def conv_block(inputs):
    x = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((2, 2, 2))(x)
    
    x = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((2, 2, 2))(x)
    
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((2, 2, 2))(x)
    
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((2, 2, 2))(x)
    
    x = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((2, 2, 2))(x)
    x = Dropout(0.5)(x)
    
    x = Flatten()(x)
    
    return x

def define_model2(num_branches):
    input_shape = (32, 32, 32, 1)
    inputs = [Input(shape=input_shape) for _ in range(num_branches)]

    branches = [conv_block(inputs[i]) for i in range(num_branches)]
    #x = Add()(branches)
    x = concatenate(branches)

    x = Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(8, activation='relu', kernel_initializer='he_uniform')(x)

    outputs = Dense(1, activation='sigmoid', kernel_initializer='he_uniform')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# try 1(0.1 diff without no overlap)

In [ ]:
def create_division(data, start, end):
    division = data.copy()
    division[(division > end)] = 1
    division = ((division - start) / (end - start)).clip(0, 1)
    return division

num_divisions = 10
start_range = 0.0
end_range = 1.0

# Training set divisions
X_train_divisions = [create_division(X_train, i * (end_range / num_divisions), 
                     (i + 1) * (end_range / num_divisions)) for i in range(num_divisions)]

# Naming the divisions as X_train_1, X_train_2, ..., X_train_10
for i in range(1, num_divisions + 1):
    locals()["X_train_" + str(i)] = X_train_divisions[i - 1]
    
# Validation set divisions
X_val_divisions = [create_division(X_val, i * (end_range / num_divisions), 
                    (i + 1) * (end_range / num_divisions)) for i in range(num_divisions)]

# Naming the divisions as X_val_1, X_val_2, ..., X_val_10
for i in range(1, num_divisions + 1):
    locals()["X_val_" + str(i)] = X_val_divisions[i - 1]
    
# Test set divisions
X_test_divisions = [create_division(X_test, i * (end_range / num_divisions), 
                   (i + 1) * (end_range / num_divisions)) for i in range(num_divisions)]
    
# Naming the divisions as X_test_1, X_test_2, ..., X_test_10
for i in range(1, num_divisions + 1):
    locals()["X_test_" + str(i)] = X_test_divisions[i - 1]
    
X_train_11= X_train.copy()
X_val_11= X_val.copy()
X_test_11= X_test.copy()

In [ ]:
branch= 11

# Build model.
with strategy.scope():
    model=define_model2(branch)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.001,
    patience=5,
    min_lr=1e-6)

checkpoint_callback = ModelCheckpoint('HU1.h5', monitor='val_accuracy',save_best_only=True)

# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=70)

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync
    
history = model.fit([X_train_1, X_train_2, X_train_3, X_train_4, X_train_5,X_train_6 ,X_train_7,X_train_8,
                     X_train_9,X_train_10,X_train_11], y_train, validation_data=([X_val_1, X_val_2, X_val_3,
                     X_val_4, X_val_5,X_val_6,X_val_7,X_val_8,X_val_9,X_val_10,X_val_11], y_val), epochs=E, 
                     batch_size=BATCH_SIZE_TPU, callbacks=[checkpoint_callback, lr_callback,early_stopping_callback],verbose=0)

# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model1 = tf.keras.models.load_model('HU1.h5')

    
loss1, accuracy1 = best_model1.evaluate([X_test_1, X_test_2, X_test_3, X_test_4, X_test_5,X_test_6,
                                         X_test_7,X_test_8,X_test_9,X_test_10,X_test_11], y_test)

print(f"accuracy: {accuracy1*100:.4f}")
print(f"loss: {loss1*100:.4f}")


In [ ]:
# Predictions
predictions = best_model1.predict([X_test_1, X_test_2, X_test_3, X_test_4, X_test_5,
                                   X_test_6, X_test_7, X_test_8, X_test_9, X_test_10, 
                                   X_test_11])

predicted_classes = (predictions > 0.5).astype(int)

# Classification Report with precision, recall, f1-score in 4 digits
class_names = ['benign', 'malignant']
report_dict = classification_report(y_test, predicted_classes, target_names=class_names, output_dict=True)

# Convert the report to a DataFrame and format the floating points
report_df = pd.DataFrame(report_dict).transpose()
report_df[['precision', 'recall', 'f1-score']] = report_df[['precision', 'recall', 'f1-score']].applymap(lambda x: f"{x:.4f}")

print("Classification Report:\n", report_df)

# Confusion Matrix
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# AUC
auc_score = roc_auc_score(y_test, predictions)
print(f"AUC: {auc_score:.4f}")

# Summarize history for accuracy
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


# try 2 (0.2 diff with no overlap)

In [ ]:
def create_division(data, start, end):
    division = data.copy()
    division[(division > end)] = 1
    division = ((division - start) / (end - start)).clip(0, 1)
    return division

num_divisions = 5
start_range = 0.0
end_range = 1.0

# Training set divisions
X_train_divisions = [create_division(X_train, i * (end_range / num_divisions), 
                     (i + 1) * (end_range / num_divisions)) for i in range(num_divisions)]

# Naming the divisions as X_train_1, X_train_2, ..., X_train_5
for i in range(1, num_divisions + 1):
    locals()["X_train_" + str(i)] = X_train_divisions[i - 1]
    
# Validation set divisions
X_val_divisions = [create_division(X_val, i * (end_range / num_divisions), 
                    (i + 1) * (end_range / num_divisions)) for i in range(num_divisions)]

# Naming the divisions as X_val_1, X_val_2, ..., X_val_5
for i in range(1, num_divisions + 1):
    locals()["X_val_" + str(i)] = X_val_divisions[i - 1]
    
# Test set divisions
X_test_divisions = [create_division(X_test, i * (end_range / num_divisions), 
                   (i + 1) * (end_range / num_divisions)) for i in range(num_divisions)]
    
# Naming the divisions as X_test_1, X_test_2, ..., X_test_5
for i in range(1, num_divisions + 1):
    locals()["X_test_" + str(i)] = X_test_divisions[i - 1]
    
X_train_6= X_train.copy()
X_val_6= X_val.copy()
X_test_6= X_test.copy()

In [ ]:
branch= 6

# Build model.
with strategy.scope():
    model=define_model2(branch)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.001,
    patience=5,
    min_lr=1e-6)

checkpoint_callback = ModelCheckpoint('HU2.h5', monitor='val_accuracy',save_best_only=True)

# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=70)

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync
    
history = model.fit([X_train_1, X_train_2, X_train_3, X_train_4, X_train_5,X_train_6], y_train, 
                    validation_data=([X_val_1, X_val_2, X_val_3, X_val_4, X_val_5,X_val_6], y_val), 
                    epochs=E, batch_size=BATCH_SIZE_TPU, callbacks=[checkpoint_callback, lr_callback,early_stopping_callback],verbose=0)

# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model2 = tf.keras.models.load_model('HU2.h5')

    
loss2, accuracy2 = best_model2.evaluate([X_test_1, X_test_2, X_test_3, X_test_4, X_test_5,X_test_6], y_test)

print(f"accuracy: {accuracy2*100:.4f}")
print(f"loss: {loss2*100:.4f}")

In [ ]:
# Predictions
predictions = best_model2.predict([X_test_1, X_test_2, X_test_3, X_test_4, X_test_5, X_test_6])
predicted_classes = (predictions > 0.5).astype(int)

# Classification Report with precision, recall, f1-score in 4 digits
class_names = ['benign', 'malignant']
report_dict = classification_report(y_test, predicted_classes, target_names=class_names, output_dict=True)

# Convert the report to a DataFrame and format the floating points
report_df = pd.DataFrame(report_dict).transpose()
report_df[['precision', 'recall', 'f1-score']] = report_df[['precision', 'recall', 'f1-score']].applymap(lambda x: f"{x:.4f}")

print("Classification Report:\n", report_df)

# Confusion Matrix
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# AUC
auc_score = roc_auc_score(y_test, predictions)
print(f"AUC: {auc_score:.4f}")

# Summarize history for accuracy
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


# try 4 (0.5 diff with no overlap)

In [ ]:
branch = 3

X_train_1 = X_train.copy()
X_train_1[X_train_1 > 0.5] = 1
X_train_1 = (X_train_1 / 0.5).clip(0, 1)

X_train_2 = X_train.copy()  # Make a copy to avoid modifying the original array
X_train_2 = ((X_train_2 - 0.5) / 0.5).clip(0, 1)
X_train_3 = X_train.copy()  



X_val_1 = X_val.copy()
X_val_1[X_val_1 > 0.5] = 1
X_val_1 = (X_val_1 / 0.5).clip(0, 1)

X_val_2 = X_val.copy()  # Make a copy to avoid modifying the original array
X_val_2 = ((X_val_2 - 0.5) / 0.5).clip(0, 1)
X_val_3 = X_val.copy()  



X_test_1 = X_test.copy()
X_test_1[X_test_1 > 0.5] = 1
X_test_1 = (X_test_1 / 0.5).clip(0, 1)

X_test_2 = X_test.copy()  # Make a copy to avoid modifying the original array
X_test_2 = ((X_test_2 - 0.5) / 0.5).clip(0, 1)
X_test_3 = X_test.copy()  


In [ ]:
# Build model.
with strategy.scope():
    model=define_model2(branch)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.001,
    patience=5,
    min_lr=1e-6)

# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=70)

checkpoint_callback = ModelCheckpoint('HU4.h5', monitor='val_accuracy',save_best_only=True)


BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync
    
history = model.fit([X_train_1, X_train_2, X_train_3], y_train, 
                    validation_data=([X_val_1, X_val_2, X_val_3], y_val), 
                    epochs=E, batch_size=BATCH_SIZE_TPU, callbacks=[checkpoint_callback, lr_callback,early_stopping_callback],verbose=0)

# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model4 = tf.keras.models.load_model('HU4.h5')

    
loss4, accuracy4 = best_model4.evaluate([X_test_1, X_test_2, X_test_3], y_test)

print(f"accuracy: {accuracy4*100:.4f}")
print(f"loss: {loss4*100:.4f}")


In [ ]:
# Predictions
predictions = best_model4.predict([X_test_1, X_test_2, X_test_3])
predicted_classes = (predictions > 0.5).astype(int)

# Classification Report with precision, recall, f1-score in 4 digits
class_names = ['benign', 'malignant']
report_dict = classification_report(y_test, predicted_classes, target_names=class_names, output_dict=True)

# Convert the report to a DataFrame and format the floating points
report_df = pd.DataFrame(report_dict).transpose()
report_df[['precision', 'recall', 'f1-score']] = report_df[['precision', 'recall', 'f1-score']].applymap(lambda x: f"{x:.4f}")

print("Classification Report:\n", report_df)

# Confusion Matrix
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# AUC
auc_score = roc_auc_score(y_test, predictions)
print(f"AUC: {auc_score:.4f}")

# Summarize history for accuracy
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
print(f"HU Accuracy (0.1 diff without overlap): {accuracy1*100:.4f}")
print(f"HU Accuracy (0.2 diff without overlap): {accuracy2*100:.4f}")
print(f"HU Accuracy (0.5 diff without overlap): {accuracy4*100:.4f}")
